In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as sci
import matplotlib.pyplot as plt
import json
from astropy.time import Time as astrotime

import tdepps.model_injection as Inj
from tdepps.model_toolkit import (create_run_dict, SignalFluenceInjector,
                                  make_rate_records, rebin_rate_rec,
                                  UniformTimeSampler, SinusFixedConstRateFunction)
from tdepps.utils import fit_spl_to_hist, arr2str

secinday = 24. * 60. * 60.
rndgen = np.random.RandomState(42439462)
print("Started: ", astrotime.now())

## Setup code

In [ ]:
def setup_data(sample_name):
    names = ["79", "86I", "86II", "86III"]

    print("# Loading experimental data arrays:")
    exp = {}
    p = "/Users/tmenne/git/phd/hese_tdepps/data/proc/offdata"
    for n in names:
        if n == sample_name or sample_name =="ALL":
            p_ = os.path.join(p, "{}.npy".format(n))
            exp[n] = np.load(p_)
            print(" - {}: '{}'".format(n, p_))

    # Seperate names for signal injector input
    exp_names = {key: exp_i.dtype.names for key, exp_i in exp.items()}


    print("# Loading Monte Carlo data arrays:")
    mc = {}
    files_ = ["IC79/IC79_corrected_MC.npy",
              "IC86_2011/IC86_corrected_MC.npy",
              "IC86_2012/IC86-2012_corrected_MC.npy",
              "IC86_2012/IC86-2012_corrected_MC.npy"]
    p = "/Users/tmenne/git/phd/hese_tdepps/data/raw"
    for n, f_ in zip(names, files_):
        if n == sample_name or sample_name =="ALL":
            p_ = os.path.join(p, f_)
            mc[n] = np.load(p_)
            print(" - {}: '{}'".format(n, p_))

    # We need "timeMJD" as a name, so change the dtype names
    for mci in mc.values():
        if "timeMJD" not in mci.dtype.names:
            idx = mci.dtype.names.index("time")
            mci.dtype.names = (mci.dtype.names[:idx] + ("timeMJD",) +
                               mci.dtype.names[idx + 1:])


    print("# Loading runlists:")
    rundict = {}
    files_ = ["IC79v24.json", "IC86_2011.json",
              "IC86_2012.json", "IC86_2013.json"]
    p = "/Users/tmenne/git/phd/hese_tdepps/data/proc/goodruns"
    for n, f_ in zip(names, files_):
        if n == sample_name or sample_name =="ALL":
            p_ = os.path.join(p, f_)
            rundict[n] = json.load(open(p_))
            print(" - {}: '{}'".format(n, p_))


    print("# Load HESE tracks locations and prepare srcs:")
    p = os.path.join("/Users/tmenne/git/phd/hese_tdepps/data/raw/",
                     "public_data_release/All_HESE_Events_4_years_tracks.txt")
    src_t, src_dec, src_ra = np.loadtxt(p, usecols=[1, 2, 3], unpack=True)
    src_ra = np.deg2rad(src_ra)
    src_dec = np.deg2rad(src_dec)

    src_arr_names = ["t", "dt0", "dt1", "ra", "dec", "w_theo"]
    types = len(src_arr_names) * [np.float]
    dtype = [(name, typ) for name, typ in zip(src_arr_names, types)]

    srcs = {}
    for key, rl in rundict.items():
        runs = rl["runs"]
        tmin = np.amin([astrotime(r["good_tstart"]).mjd for r in runs])
        tmax = np.amax([astrotime(r["good_tstop"]).mjd for r in runs])
        t_mask = (src_t >= tmin) & (src_t <= tmax)

        src_ti = src_t[t_mask]
        nsrcs_i = len(src_ti)

        srcs_i = np.empty((nsrcs_i, ), dtype=dtype)

        srcs_i["t"] = src_ti
        # Leave empty for now, time window is set explicitely below
        srcs_i["dt0"] = np.zeros(nsrcs_i, dtype=np.float)
        srcs_i["dt1"] = np.zeros(nsrcs_i, dtype=np.float)
        srcs_i["ra"] = src_ra[t_mask]
        srcs_i["dec"] = src_dec[t_mask]
        srcs_i["w_theo"] = np.ones(nsrcs_i, dtype=np.float)
        srcs[key] = srcs_i

    p = ("/Users/tmenne/git/phd/hese_tdepps/data/proc/" +
         "time_windows/time_windows.txt")
    time_window = np.loadtxt(fname=p)[tw_id]
    print("   + Loaded time window list: '{}'".format(p))
    print("   + Using time window {:2d}: {:.2f}s".format(tw_id,
                                                       np.diff(time_window)[0]))
    for key, srcs_i in sorted(srcs.items()):
        print(" - {}:".format(key))
        srcs_i["dt0"] = np.repeat(time_window[0], repeats=len(srcs_i))
        srcs_i["dt1"] = np.repeat(time_window[1], repeats=len(srcs_i))
        for n in srcs_i.dtype.names:
            print("   + {}: [{}]".format(n, arr2str(srcs_i[n], fmt="{:.2f}")))
            
    return exp, exp_names, mc, srcs, rundict

## Prepare

In [ ]:
# Loading data setup
tw_id = 20
sample_name = "86I"  # One of: 79, 86I, 86II, 86III or ALL
if sample_name not in ["79", "86I", "86II", "86III", "ALL"]:
    raise ValueError("Wrong name for `n` chosen.")

exp, exp_names, mc, srcs, rundict = setup_data(sample_name)

# Single year aliases for testing
if sample_name is not "ALL":
    exp_ = exp[sample_name]
    MC_ = mc[sample_name]
    srcs_ = srcs[sample_name]
    run_dict_ = create_run_dict(rundict[sample_name]["runs"])

print("\nDone, loaded sample(s) for '{}'.".format(sample_name))

## Start

In [ ]:
# Finer resolution around the horizon region, where we usually switch the event
# selections from northern to southern samples
hor = np.sin(np.deg2rad(30))
sindec_bins = np.unique(np.concatenate([
                        np.linspace(-1., -hor, 3 + 1),    # south
                        np.linspace(-hor, +hor, 14 + 1),  # horizon
                        np.linspace(+hor, 1., 3 + 1),     # north
                        ]))
rate_rebins = np.linspace(exp_["timeMJD"].min(), exp_["timeMJD"].max(), 12)

# Choose spl_s so that the spline sticks a little more to the data
bg_inj_args = {"sindec_bins": sindec_bins, "rate_rebins": rate_rebins,
               "spl_s": len(sindec_bins) // 2}
grb_inj = Inj.GRBModelInjector(bg_inj_args=bg_inj_args, rndgen=rndgen)

In [ ]:
recs = make_rate_records(T=exp_["timeMJD"], run_dict=run_dict_)
rates, new_bins, stddev, _ = rebin_rate_rec(rate_rec=recs, bins=rate_rebins,
                                     ignore_zero_runs=True)
mids = 0.5 * (new_bins[:-1] + new_bins[1:])

rate_func = SinusFixedConstRateFunction(p_fix=365.)
allsky_res = rate_func.fit(rate=rates, srcs=srcs_, t=mids, w=1. / stddev)

t0_fix = allsky_res.x[1]
print("Best fit t0 before first event: ",
      t0_fix - exp_["timeMJD"].min(), "days")

rate_func = SinusFixedConstRateFunction(p_fix=365., t0_fix=t0_fix)

In [ ]:
def signal_flux_model(trueE, E0=1., gamma=2.):
    return (trueE / E0)**(-gamma)

ts = UniformTimeSampler(random_state=None)

sig_inj = SignalFluenceInjector(signal_flux_model, time_sampler=ts)
sig_inj.fit(srcs_, MC=MC_, exp_names=["timeMJD", "dec", "ra", "sigma", "logE"])

In [ ]:
grb_inj.fit(X=exp_, srcs=srcs_, run_dict=run_dict_, sig_inj=sig_inj)

### BG splines and sampling

In [ ]:
x = np.linspace(-1, 1, 100)
bins = grb_inj._sin_dec_bins
mids = 0.5 * (bins[:-1] + bins[1:])
print("Allsky best params: " + arr2str(grb_inj._allsky_pars))

for n in ["amp", "base"]:
    fig, ax = plt.subplots(1, 1)
    spl = grb_inj._param_splines[n]
    vals = grb_inj._best_pars[n]
    err = np.copy(grb_inj._best_stddevs[n])
   
    ax.plot(mids, grb_inj._best_pars[n], color="C7", ls="--")
    ax.errorbar(mids, vals, yerr=err, fmt="o", color="C1")
    ax.plot(x, spl(x), color="k")

    if n == "amp":
        ax.axhline(0, 0, 1, color="C7", ls="--")
    else:
        ax.set_ylim(0, None)

    ax.set_xlabel("sindec")
    ax.set_ylabel(n)
    ax.set_title("Integral: {:.3f} mHz".format(spl.integral(-1, 1) * 1e3))
    # Show sindec bin borders
    ylim = ax.get_ylim()
    ax.vlines(sindec_bins, ylim[0], ylim[1],
               linestyles=":", colors="C7")
    ylim = ax.set_ylim(ylim)

    plt.show()

In [ ]:
x = np.linspace(-1, 1, 200)
bins = 100
plt.vlines(sindec_bins, 0, 1, linestyles="--", colors="C7", zorder=-1)
plt.hist(np.sin(exp_["dec"]), bins=bins, density=True)
plt.plot(x, grb_inj._data_spl(x))

plt.show()

In [ ]:
# Make multiple trials and concat to compare to spline with large stats
# Use internal debug var to get samples per source to compare to splines
sam = [list() for _ in range(len(grb_inj.srcs))]
for _ in range(200):
    _ = grb_inj.get_sample()
    for i, sami in enumerate(grb_inj._bg_cur_sam):
        sam[i].append(sami)
        
sam = [np.concatenate(sami) for sami in sam]

In [ ]:
x = np.linspace(-1, 1, 200)
bins = 40
# bins = np.linspace(-1, 1, 17 + 1)
for j, sami in enumerate(sam):
    plt.vlines(sindec_bins, 0, 1, linestyles="--", colors="C7", zorder=-1)
    # Plot allyear sample for comparison
    plt.hist(np.sin(exp_["dec"]), bins=bins, density=True, color="0.75")
    plt.plot(x, grb_inj._data_spl(x), color="C0", ls=":", lw=3)
    # Drawn sample per source. Red hist should approx. follow black spline
    plt.hist(np.sin(sami["dec"]), bins=bins, density=True,
             histtype="step", lw=2.5, color="C3")
    plt.plot(x, grb_inj._sin_dec_splines[j](x), color="k")

    plt.show()

### MC Injector

In [ ]:
# Testing signal injector
grb_inj.get_sample(n_signal=10000)
sig_sam = grb_inj._sig_cur_sam

In [ ]:
# Sampled MC hist
n = "timeMJD"
_ = plt.hist(sig_sam[n], density=True, bins=100)
if n != "timeMJD":
    plt.vlines(grb_inj.srcs[n], 0, 1)
    plt.yscale("log", nonposy="clip")
    plt.show()
else:
    for src_idx
    plt.vlines(grb_inj.srcs["t"], 0, 1)
    plt.vlines(grb_inj.srcs["t"] + grb_inj.srcs["dt0"] / 86400.,
               0, 1, linestyles="--")
    plt.vlines(grb_inj.srcs["t"] + grb_inj.srcs["dt1"] / 86400.,
               0, 1, linestyles="--")
    plt.show()

In [ ]:
# Compare to full MC pool distribution
smc = sig_inj._MC
_ = plt.hist(smc["dec"], bins=100, density=True,
             weights=smc["trueE"]**-2*smc["ow"])
if n in ["ra", "dec"]:
    plt.vlines(grb_inj.srcs[n], 0, 2)
plt.yscale("log", nonposy="clip")
plt.show()

### Test LLH scan instead of hess_inv from fitres

In [ ]:
def plot_llh_scan(bfs, stds, llh, grid):
    """
    Plot the llh scan with errors and contours
    
    Parameters
    ----------
    bfs : array-like, shape (2)
        Best fit result parameters, around which the LLH was scanned.
    stds : array-like, shape (2)
        Approximate standard deviations (symmetric) for each fit parameter,
        obtained using Wilks' theorem on the scanned space.
    llh : array-like, shape (nbins, nbins)
        Scanned LLH values.
    grid : list
        X, Y grid, same shape as ``llh``.
    """
    bf_x, bf_y = bfs
    std_x, std_y = stds
    x, y = grid
    
    # Plot scan
    fig, ax = plt.subplots(1, 1)
    img = ax.pcolormesh(x, y, llh)
    fig.colorbar(img, ax=ax)

    # Plot 1, 2, 3 sigma contours
    vals = np.amin(llh) - scs.chi2.logsf(df=2, x=[1**2, 2**2, 3**2])
    ax.contour(x, y, llh, vals, linestyles=["--", "-.", "--"], colors="w")
    
    # Plot best fit with symmetric errors
    ax.errorbar(bf_x, bf_y, xerr=std_x, yerr=std_y, fmt="o", c="w", capsize=5)   
    
    ax.xlabel = ("amplitude")
    ax.ylabel = ("baseline")

def get_stddev_from_scan(func, args, bfs, rngs, nbins=50):
    """
    Scan the rate_func chi2 fit LLH to get stddevs for the best fit params a, d.
    Using matplotlib contours and averaging to approximately get the variances.
    Note: This is not a true LLH profile scan in both variables.
    
    Parameters
    ----------
    func : callable
        Loss function to be scanned, used to obtain the best fit. Function
        is called as done with a scipy fitter, ``func(x, *args)``.
    args : tuple
        Args passed to the loss function ``func``. For a rate function, this is
        ``(mids, rates, weights)``.
    bfs : array-like, shape (2)
        Best fit result parameters.
    rngs : list
        Parameter ranges to scan: ``[bf[i] - rng[i], bf[i] + rng[i]]``.
    nbins : int, optional
        Number of bins in each dimension to scan. (Default: 100)
        
    Returns
    -------
    stds : array-like, shape (2)
        Approximate standard deviations (symmetric) for each fit parameter,
        obtained using Wilks' theorem on the scanned space.
    llh : array-like, shape (nbins, nbins)
        Scanned LLH values.
    grid : list
        X, Y grid, same shape as ``llh``.
    """
    def _scan_llh(bf_x, rng_x, bf_y, rng_y):
        """ Scan LLH and return contour vertices """
        x_bins = np.linspace(bf_x - rng_x, bf_x + rng_x, nbins)
        y_bins = np.linspace(bf_y - rng_y, bf_y + rng_y, nbins)   
        x, y = np.meshgrid(x_bins, y_bins)
        AA, DD = map(np.ravel, [x, y])
        llh = np.empty_like(AA)
        for i, (ai, di) in enumerate(zip(AA, DD)):
            llh[i] = func((ai, di), *args)
        llh = llh.reshape(x.shape)
        # Get the contour points and average over min, max per parameter
        one_sigma_level = np.amin(llh) - scs.chi2.logsf(df=2, x=[1**2])

        # https://stackoverflow.com/questions/5666056
        cntr = plt.contour(x, y, llh, one_sigma_level)
        plt.close("all")
        paths = [lcol.vertices for lcol in cntr.collections[0].get_paths()]
        # Call undocumented base of plt.contour, to avoid creating a figure.
        # Not working for mpl 2.2.2 any more, because _cntr was deleted.
        # cntr = contour.Cntr(x, y, llh)
        # paths = cntr.trace(level0=one_sigma_level)
        # paths = paths[:len(paths) // 2]  # First half of list has the vertices
        return paths, llh, [x, y]

    def _is_path_closed(paths, rng_x, rng_y):
        """
        We want the contour to be fully contained. Means there is only one path
        and the first and last point are close together.
        Returns ``True`` if contour is closed.
        """
        closed = False
        if len(paths) == 1:
            vertices = paths[0]
            # If no contour is made, only 1 vertex is returned -> invalid
            if len(vertices) > 1:
                max_bin_dist = np.amax([rng_x / float(nbins),
                                        rng_y / float(nbins)])
                closed = np.allclose(vertices[0], vertices[-1],
                                     atol=max_bin_dist, rtol=0.)
        return closed
    
    def _get_stds_from_path(path):
        """ Create symmetric stddevs from the path vertices """
        x, y = path[:, 0], path[:, 1]
        # Average asymmetricities in both direction
        x_min, x_max = np.amin(x), np.amax(x)
        y_min, y_max = np.amin(y), np.amax(y)
        return 0.5 * (x_max - x_min), 0.5 * (y_max - y_min)
           
    # Scan the LLH, adapt scan range if contour is not closed
    bf_x, bf_y = bfs
    rng_x, rng_y = rngs
    closed = False
    while not closed:
        # Default is scaling up, when range is too small
        scalex, scaley = 10., 10.
        # Get contour from scanned LLH space
        paths, llh, grid = _scan_llh(bf_x, rng_x, bf_y, rng_y)
        closed = _is_path_closed(paths, rng_x, rng_y)       
        if closed:
            vertices = paths[0]
            # Estimate scale factors to get contour in optimum resolution
            diffx = np.abs(np.amax(vertices[:, 0]) - np.amin(vertices[:, 0]))
            diffy = np.abs(np.amax(vertices[:, 1]) - np.amin(vertices[:, 1]))
            scalex = diffx / rng_x
            scaley = diffy / rng_y
            # Contour can be closed, but extremely zoomed out in only one param
            if not np.allclose([scalex, scaley], 1., atol=0.5, rtol=0.):
                print("Contour is very distorted in one direction")
                closed = False
            else:
                # Rescan valid contour to use optimal scan resolution
                for i in range(2):
                    std_x, std_y = _get_stds_from_path(vertices)
                    rng_x = std_x * 1.05  # Allow a little padding
                    rng_y = std_y * 1.05
                    paths, llh, grid = _scan_llh(bf_x, rng_x, bf_y, rng_y)
                    # Recheck if path is still valid
                    closed = _is_path_closed(paths, rng_x, rng_y)
        # Must be seperated if, because path can get invalid in rescaling step
        if not closed:
            print("Open or no contour, rescale")
            rng_x *= scalex
            rng_y *= scaley

    vertices = paths[0]
    stds = np.array(_get_stds_from_path(vertices))
    return stds, llh, grid

In [ ]:
sindec = exp_["sinDec"]
t_ = np.linspace(exp_["timeMJD"].min(), exp_["timeMJD"].max(), 200)

allres = []
errs = []
for j, (lo, hi) in enumerate(zip(sindec_bins[:-1], sindec_bins[1:])[:]):
    mask = (sindec >= lo) & (sindec <= hi)

    recs = make_rate_records(T=exp_["timeMJD"][mask], run_dict=run_dict_)
    rates, new_bins, stddev, _ = rebin_rate_rec(rate_rec=recs, bins=rate_rebins,
                                                ignore_zero_runs=True)
    new_mids = 0.5 * (new_bins[:-1] + new_bins[1:])
    weights = 1. / stddev
    res = rate_func.fit(rate=rates, srcs=srcs_, t=new_mids, w=weights)
    bfs = np.array([res.x[0], res.x[1]])
    allres.append(res)
    
    plt.errorbar(recs["start_mjd"], recs["rate"], yerr=recs["rate_std"],
                 fmt=",", alpha=0.2, color="C0")
    plt.plot(recs["start_mjd"], recs["rate"], marker=".", ls="", color="C0")
    plt.plot(new_bins, np.r_[rates[0], rates], drawstyle="steps-pre", color="k")
    plt.plot(t_, rate_func.fun(t=t_, pars=bfs), color="C3")
    plt.ylim(0, 3. * bfs[1])
    plt.show()
    
    # Empirical seed estimates for amplitude and baseline scan range
    args = (new_mids, rates, weights)
    rngs = np.array([bfs[0], bfs[1] / 10.])
    stds, llh, grid = get_stddev_from_scan(
        func=rate_func._lstsq, args=args, bfs=bfs, rngs=rngs, nbins=20)
    
    plot_llh_scan(bfs, stds, llh, grid)
    plt.show()
    
    errs.append(stds)

errs = np.array(errs)

In [ ]:
# 0 = amp, 1 = base
# Note: The spline is not renormalized here, so there might be differences in
#       scale to the one from the module
for idx in [0, 1]:
    x = np.linspace(-1, 1, 100)
    mids = 0.5 * (sindec_bins[:-1] + sindec_bins[1:])
    norm = np.diff(sindec_bins)

    vals = np.array([res.x[idx] for res in allres]) / norm
    err_ = errs.T[idx] / norm

    # Prepare for spl fit
    w = 1. / err_
    spl, vals, pts, w = fit_spl_to_hist(h=vals, bins=sindec_bins, w=w, s=10)
    
    plt.plot(pts, vals, color="C7", ls="--")
    plt.errorbar(pts, vals, yerr=1. / w, fmt="o", color="C1")
    plt.plot(x, spl(x), color="k")
    plt.title("Integral: {:.3f} mHz".format(spl.integral(-1, 1) * 1e3))

    if idx == 0:
        plt.axhline(0, 0, 1, color="C7", ls="--")
        plt.ylabel("amp")
    else:
        plt.ylim(0, None)
        plt.ylabel("base")
    plt.xlabel("sindec")
 
    plt.show()